# Bridge Detection

In [2]:
%load_ext autoreload
%autoreload 2

import json
import os
import sys
sys.path.insert(0, '/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset')
import header as head

bridges_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges"
type2_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 2"

1. Nodes that have followers from both communities. 
2. Nodes that follows other communites but only has follower from one commmunity

**code preparation**

In [ ]:
# Obtained who followed by whom relationship
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as f:
    follows = json.load(f)

who_followed_by_whom = {}
# Find who followed by whom
for user in follows:
    who_followed_by_whom[user] = []
    followed_by = who_followed_by_whom[user]

    for f in follows:
        if user in follows[f]: followed_by.append(f)

# head.writeToJSON(head.getRoot(),"who_followed_by_whom",who_followed_by_whom)

In [ ]:
# obtain all followers from both communities per user 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as f:
    communities = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as f:
    follows = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who_followed_by_whom.json") as f:
    followed_by = json.load(f)

rumours_set = communities["rumours"]
non_rumours_set = communities["non_rumours"]
uncategorized_set = communities["uncategorized"]

bridges = {}
ids = []
for c in communities.keys():
    ids.extend(communities[c])


for id in ids:
    try:
        followers = followed_by[id]
    except: 
        continue
    
    bridges[id] = {
        "rumours" : [],
        "non_rumours" : [],
        "uncategorized" : []
    }
    node = bridges[id]

    for followee in followers:
        if followee in rumours_set: node["rumours"].append(followee)
        elif followee in non_rumours_set: node["non_rumours"].append(followee)
        elif followee in uncategorized_set: node["uncategorized"].append(followee)
    
    # uncomment the following if statement to obtain "user_followed_by_unfiltered" file
    if (len(node["rumours"])==0 and len(node["non_rumours"])==0
        and len(node["uncategorized"])==0): bridges.pop(id)

head.writeToJSON(bridges_path,"user_followed_by",bridges)

We test whether type 3 bridges affect the identification of type 1 bridge. 

Result: type 3 bridges does not affect the amount of type 1 bridges being identified

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/bridges.json") as file:
    b1 = json.load(file)
    b1 = b1.keys()

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/bridges_v2.json") as file:
    b2 = json.load(file)
    b2 = b2.keys()

print(b1==b2)

Update user_followed_by with additional information about bridges

In [15]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by.json") as file:
    followed_by = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/bridges.json") as file:
    bridges = json.load(file)
    bridge_ids = bridges.keys()

new_followed_by = {}

for f in followed_by:
    user = followed_by[f]
    new_followed_by[f] = {
        "rumours":[],
        "non_rumours":[],
        "uncategorized":[],
        "bridges":[]
    }

    for id in user["rumours"]:
        if id in bridge_ids : (new_followed_by[f])["bridges"].append(id)
        else : (new_followed_by[f])["rumours"].append(id)
    
    for id in user["non_rumours"]:
        if id in bridge_ids : (new_followed_by[f])["bridges"].append(id)
        else : (new_followed_by[f])["non_rumours"].append(id)

    for id in user["uncategorized"]:
        if id in bridge_ids : (new_followed_by[f])["bridges"].append(id)
        else : (new_followed_by[f])["uncategorized"].append(id)

head.writeToJSON(bridges_path,"user_followed_by",new_followed_by)

## Type 1 Bridge Analysis

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by_v2.json") as file:
    followed_by = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as file:
    follows = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/with_bridges.json") as file:
    communities = json.load(file)

type1 = {}
for id in followed_by:
   user = followed_by[id]
   if len(user["rumours"]) > 0 and len(user["non_rumours"]) > 0:
       type1[id] = {}
       (type1[id])["followed_by"] = followed_by[id]
       (type1[id])["follows"] = {
            "rumours":[],
            "non_rumours":[],
            "bridges":[]
        }

for id in type1:
    bridge = type1[id]
    following = follows[id]

    for f in following:
        if f in communities["rumours"]: ((type1[id])["follows"])["rumours"].append(f)
        elif f in communities["non_rumours"]: ((type1[id])["follows"])["non_rumours"].append(f)
        elif f in communities["bridges"]: ((type1[id])["follows"])["bridges"].append(f)



head.writeToJSON(bridges_path,"type1",type1)


**Checks whether all type1 bridges tweeted source tweets**  
Result: From the analysis below, we can be certain that all type1 bridges tweeted source tweet.

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/tweetId_annotations.json") as file:
    src_tweets = json.load(file)
    src_tweets = (src_tweets["Source_Tweets"]).keys()

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/Id-conversions/tweet_to_user.json") as file:
    tweet_to_user = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type1.json") as file:
    type1 = json.load(file)
    type1 = type1.keys()

src_ids = []
for src in src_tweets:
    src_ids.append(tweet_to_user[src])

for t in type1:
    if t in src_ids : print(t)



**Checks where does all type1 bridges originate from**  
Either from rumour or non-rumour community.

Results: 
- All type1 bridges originated from rumour community.
- We can also argue that all type1 bridges spread rumours to non-rumour community


In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as file:
    communities = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/with_bridges.json") as file:
    bridges = json.load(file)
    bridges = bridges["bridges"]

from_rumour = []
from_non_rumour = []

for b in bridges:
    if b in communities["rumours"]: from_rumour.append(b)
    elif b in communities["non_rumours"]: from_non_rumour.append(b)

print("from_rumour")
for f in from_rumour:
    print(f)

print("from_non_rumour")
for f in from_non_rumour:
    print(f)


Obtaining centrality information regarding type 1 bridges' followers

In [6]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 1/type1.json") as file:
    type1 = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/betweeness/directed_betweenness.json") as file:
    betweeness = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/closeness/rumour_closeness.json") as file:
    rum_closeness = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/closeness/non_rumour_closeness.json") as file:
    nonrum_closeness = json.load(file)

type1_centrality = {}

for id in type1:
    bridge = type1[id]
    type1_centrality[id] = {
        "rum_count" : len((bridge["followed_by"])["rumours"]),
        "nonrum_count" : len((bridge["followed_by"])["non_rumours"]),
        "rumours" : {},
        "non_rumours" : {},
    }

    rum_followers = (type1_centrality[id])["rumours"]
    nonrum_followers = (type1_centrality[id])["non_rumours"]

    for user in (bridge["followed_by"])["rumours"]:
        betw = (betweeness["rumours"])[user]
        close = (rum_closeness["rumours"])[user]
        if betw > 0 or close > 0:
            rum_followers[user] = {
                "betweeness":(betweeness["rumours"])[user],
                "closeness":(rum_closeness["rumours"])[user]
            }
    
    for user in (bridge["followed_by"])["non_rumours"]:
        betw = (betweeness["non_rumours"])[user]
        close = (nonrum_closeness["non_rumours"])[user]
        if betw > 0 or close > 0:
            nonrum_followers[user] = {
                "betweeness":(betweeness["non_rumours"])[user],
                "closeness":(nonrum_closeness["non_rumours"])[user]
            }

head.writeToJSON(head.makePath([bridges_path,"type 1"]),"centrality_2",type1_centrality)

## Type 2 Bridge Analysis

Obtain all users whom followers are all from the same community

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by_v2.json") as file:
    followed_by = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as file:
    follows = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/with_bridges.json") as file:
    communities = json.load(file)

type2 = {}
for id in followed_by:
    user = followed_by[id]
    if len(user["rumours"]) == 0 or len(user["non_rumours"]) == 0:
        if len(user["rumours"]) == 0 and len(user["non_rumours"]) == 0:
            continue
        
        else :
            type2[id] = {}
            (type2[id])["followed_by"] = followed_by[id]
            (type2[id])["follows"] = {
                "rumours":[],
                "non_rumours":[],
                "bridges":[]
            }

for id in type2:
    bridge = type2[id]
    following = follows[id]

    for f in following:
        if f in communities["rumours"]: ((type2[id])["follows"])["rumours"].append(f)
        elif f in communities["non_rumours"]: ((type2[id])["follows"])["non_rumours"].append(f)
        elif f in communities["bridges"]: ((type2[id])["follows"])["bridges"].append(f)


head.writeToJSON(type2_path,"general",type2)

Type 2 bridges that follows another community from its own followers

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 2/general.json") as file:
    gen_type2 = json.load(file)

follows_other_community = {}
follows_other_community.clear()

for type2 in gen_type2:
    bridge = gen_type2[type2]
    followees = bridge["followed_by"]
    following = bridge["follows"]
    if len(followees["rumours"])!=0 and len(following["non_rumours"]) !=0: 
        follows_other_community[type2] = gen_type2[type2]
    elif len(followees["non_rumours"])!=0 and len(following["rumours"]) !=0: 
        follows_other_community[type2] = gen_type2[type2]
    # else: follows_other_community[type2] = gen_type2[type2]

head.writeToJSON(type2_path,"follows_other_community",follows_other_community)

Type 2 bridges that follows other bridges
Result:
- Most type 2 bridges only have followers from rumour and bridge communities. 
- This shows that the traverse of information from rumour to non-rumour community and vice versa is only acheived by type 1 bridges.

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 2/general.json") as file:
    gen_type2 = json.load(file)

follows_bridge = {}
for type2 in gen_type2:
    bridge = gen_type2[type2]
    following = bridge["follows"]
    if len(following["bridges"]) != 0:
        # non-relaxed if statement
        if (len(following["rumours"]) == 0 and len(following["non_rumours"]) == 0):
            follows_bridge[type2] = gen_type2[type2]

head.writeToJSON(type2_path,"follows_bridges",follows_bridge)